In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
#import matplotlib.pyplot as plt

In [30]:
data = pd.read_csv('ER.csv', names=['PD','ERL','loan_amnt','int_rate','ER', 'target'])
state_abbrev=pd.read_csv('state_abbrev.csv')
state_abbrev = [tuple(x) for x in state_abbrev.values]
states = pd.read_csv('states.csv', names=state_abbrev)

In [3]:
N_loans=len(data.index)
loan_ID = list(range(0,N_loans))
state_ID = list(range(0,states.shape[1]))

In [4]:
# Create an empty model
m = Model('portfolio')

Academic license - for non-commercial use only


In [5]:
# Add a variable for each loan
vars = pd.Series(m.addVars(loan_ID, vtype=GRB.INTEGER), index=loan_ID)
#vars.head()

In [6]:
# Objective is to maximize expected return ER.
portfolio_ER = vars.dot(data['ER'])
m.setObjective(portfolio_ER, GRB.MAXIMIZE)

In [7]:
m.addConstr(vars.sum() == 5000000, 'budget')

<gurobi.Constr *Awaiting Model Update*>

In [8]:
m.addConstrs((vars[i] <= data['loan_amnt'][i] for i in range(N_loans)), name='loan_size')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [9]:
#m.addConstrs((vars.sum(i)<500000 for i in AZ_s), 'diversification')

In [11]:
#m.addConstrs((vars.dot(states.iloc[:,i]) for i in state_ID), 'diversification')
CO_df=states[(states['CO'].select_dtypes(include=['number']) != 0).any(1)]
CO_s = CO_df.index.values
CO_l = list(CO_s)
m.addConstr((vars.iloc[CO_l].sum() <= 500000), 'diversification')
#vars[[12]]

<gurobi.Constr *Awaiting Model Update*>

In [12]:
#for i in state_ID:
#    print(states.iloc[:,i].dot(data['loan_amnt']))

In [13]:
#print(states.as_matrix[1,2])
#data.loc[:,'loan_amnt']
#for i in state_abbrev: print(states.loc[:,i[0]] as.matrix(states)[:,i] %*% data.loc[:,'loan_amnt'] )
#for i in state_abbrev: print(states.loc[:,i[0]].dot(loan_ID))
#for i in state_abbrev: print(states.loc[:,i[0]].dot(data.loc[:,'loan_amnt']))
#states.loc[:,'AZ'].tolist()
#for i in state_abbrev: print(states.loc[:,i[0]])
#vars[1].x
#type(vars[i])


#list(AZ_df.index.values)
#vars[i].sum()<500000 for i in AZ_df

#loan_ID[(states['AZ'].select_dtypes(include=['number']) != 0).any(1)]
#AZ_df.head()
#AZ_df.index[0]
#ls = []
#ls.extend(AZ_df.index[0])

In [14]:
#m.addConstrs((vars[i] <= 500000 for i in range(N_loans)), name='diversification')
#m.addConstrs((vars.dot(states.loc[:,i[0]])<=500000 for i in state_abbrev), name='diversification')
#type(data['loan_amnt'])
#data.loc[:,'loan_amnt'].head()
#states.loc[:,'AZ'].head()
#type(data.loc[1,'loan_amnt'])
#type(states.loc[1,'AZ'][0])

#type(data.loc[1,'loan_amnt']*states.loc[1,'AZ'][0])

#data.loc[:,'loan_amnt'].dot(states.loc[:,'AZ'])

#list(range(0,states.shape[1]))

#data.loc[:,'loan_amnt'].dot(states.loc[:,'AZ']) for i in state_abbrev



#print(states.iloc[:,2])

#type(data['loan_amnt'][3])
#type(states.loc[:,'AZ'])
#len(states.loc[:,'AZ'])
#states.loc[:,'AZ'].dot(data['loan_amnt'])
#states.loc[:,'AZ']
#type(N_loans)
m.update()

In [15]:
#states.loc[:,'AZ']
#states.head()


In [16]:

#   print(states.loc[:,i[0]])
    #print("{} {} {}".format(i[0], "total is" , 1))
    
    #m.update()
#m.reset()


In [17]:
#m.printStats()

In [18]:
# Optimize model to find the minimum risk portfolio
m.optimize()

Optimize a model with 96781 rows, 96779 columns and 206923 nonzeros
Variable types: 0 continuous, 96779 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-05, 3e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 5e+06]
Presolve removed 96779 rows and 0 columns (presolve time = 140s) ...
Presolve removed 96779 rows and 8177 columns (presolve time = 276s) ...
Presolve removed 96779 rows and 83389 columns
Presolve time: 277.18s
Presolved: 2 rows, 13390 columns, 26546 nonzeros
Variable types: 0 continuous, 13390 integer (0 binary)
Found heuristic solution: objective 1342843.5042
Found heuristic solution: objective 1440764.4804

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5941674e+06   1.218875e+05   0.000000e+00    278s
       2    1.4448936e+06   0.000000e+00   0.000000e+00    278s

Root relaxation: objective 1.444894e+06, 2 iterations, 0.06 seconds

    Nodes    |    Cur

In [19]:
print('Objective: %g' % m.objVal)
count=0
o=0
variables=[0] * N_loans
for v in m.getVars():
    if v.x>0:
        variables[o]=v.x
        print('%s %g %g' % (v.varName, o, v.x))
        count=count+1
    o=o+1
 
print("{} {} {} {}".format(count, "investments out of total of", o, "opportunities"))
data['inv']=variables
#data.head()
#pos_investment = data['inv'] >0
#data[pos_investment]

Objective: 1.44489e+06
C807 807 16800
C1773 1773 14000
C2128 2128 12800
C2473 2473 35000
C3060 3060 5000
C3512 3512 12000
C4228 4228 20725
C4854 4854 27075
C5946 5946 12000
C6090 6090 12000
C6239 6239 16000
C8128 8128 16800
C8474 8474 35000
C9129 9129 11375
C9636 9636 35000
C9729 9729 28000
C9949 9949 20000
C10242 10242 40000
C11101 11101 4200
C11139 11139 30000
C11244 11244 19200
C11324 11324 21000
C11706 11706 11075
C11840 11840 25000
C12016 12016 15225
C12083 12083 26900
C12116 12116 20000
C13086 13086 16800
C13524 13524 24000
C13828 13828 6000
C14634 14634 21000
C15088 15088 4700
C15559 15559 12000
C15604 15604 24000
C16029 16029 10400
C16826 16826 30000
C16887 16887 20000
C17258 17258 18575
C17285 17285 35000
C17477 17477 11200
C18083 18083 30000
C18128 18128 12000
C18420 18420 14000
C18480 18480 24000
C18735 18735 5125
C19344 19344 30000
C21666 21666 35000
C21792 21792 16000
C22069 22069 30000
C22813 22813 10000
C22994 22994 22200
C23435 23435 38400
C23969 23969 15000
C24005 2400

In [20]:
for i in state_ID:
    print(state_abbrev[i], states.iloc[:,i].dot(data.loc[:,'inv']))

('AL',) 0.0
('AZ',) 0.0
('AR',) 16000.0
('CA',) 223725.0
('CO',) 500000.0
('CT',) 176000.0
('DE',) 85950.0
('FL',) 0.0
('GA',) 0.0
('HI',) 210025.0
('ID',) 183325.0
('IL',) 0.0
('IN',) 25000.0
('IA',) 313500.0
('KS',) 21925.0
('KY',) 51000.0
('LA',) 16000.0
('ME',) 135525.0
('MD',) 156675.0
('MA',) 191750.0
('MI',) 30675.0
('MN',) 164575.0
('MS',) 44000.0
('MO',) 40925.0
('MT',) 41000.0
('NE',) 30325.0
('NV',) 94700.0
('NH',) 0.0
('NJ',) 0.0
('NM',) 0.0
('NY',) 151950.0
('NC',) 39200.0
('ND',) 48800.0
('OH',) 455575.0
('OK',) 180275.0
('OR',) 91775.0
('PA',) 57000.0
('RI',) 240150.0
('SC',) 40000.0
('SD',) 42200.0
('TN',) 0.0
('TX',) 48000.0
('UT',) 469150.0
('VT',) 0.0
('VA',) 211800.0
('WA',) 0.0
('WV',) 160325.0
('WI',) 0.0
('WY',) 11200.0


In [39]:
#sum(variables)
#variables
#data['inv']=variables
#sum(data['inv'])
#data['inv']=variables
sum(data['inv'])
#sum(pos_investment)
#test_crit=data['ER']>0.305
#data[test_crit].head(10)

5000000.0

In [34]:
data['RR'] = data['int_rate']-data['target']*0.4

In [22]:
#data['investment']=vars
#len(data[pos_investment])
#data[pos_investment].head()
#data.head()
#len(vars)
#for v in vars: data['investment']=v.x
#data.describe()

In [35]:
np.savetxt("Logistic_portfolio.csv", data, delimiter=",")

In [24]:
#avg_ret=data['loan_amnt'].dot(data['int_rate'])/sum(data['loan_amnt'])
#sum(data['inv'])
#log_ret=data['investment'].dot(data['int_rate'])/sum(data['investment'])

In [25]:
ret = data['loan_amnt'].dot(data['int_rate'])/sum(data['loan_amnt'])
print("LendingClub loan portfolio return assuming no bankruptcies {0:.2f} %".format(ret*100))

LendingClub loan portfolio return assuming no bankruptcies 13.81 %


In [47]:
ret = data['loan_amnt'].dot(data['RR'])/sum(data['loan_amnt'])
print("LendingClub loan portfolio realized return {0:.2f} %".format(ret*100))

LendingClub loan portfolio realized return 10.87 %


In [45]:
ret = data['loan_amnt'].dot(data['ER'])/sum(data['loan_amnt'])
print("LendingClub loan portfolio estimated return based on our model {0:.2f} %".format(ret*100))

LendingClub loan portfolio estimated return based on our model 10.26 %


In [46]:
ret = data['inv'].dot(data['ER'])/sum(data['inv'])
print("Our portfolio estimated return {0:.2f} %".format(ret*100))

Our portfolio estimated return 28.90 %


In [40]:
ret = data['inv'].dot(data['RR'])/sum(data['inv'])
print("Our portfolio realized return {0:.2f} %".format(ret*100))

Our portfolio realized return 21.26 %


In [42]:
data.head()

,PD,ERL,loan_amnt,int_rate,ER,target,RR,inv
0,0.004935,0.001974,15000.0,0.0532,0.051226,0.0,0.0532,0.0
1,0.109972,0.043989,17000.0,0.0749,0.030911,0.0,0.0749,0.0
2,0.038825,0.015530,20000.0,0.0532,0.037670,0.0,0.0532,0.0
3,0.040959,0.016384,16000.0,0.1274,0.111016,0.0,0.1274,0.0
4,0.159881,0.063952,2000.0,0.1699,0.105948,0.0,0.1699,0.0
